# Sprint ディープラーニングフレームワーク
# １．コードリーディング
TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。                      
tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。

# 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。                  
* 重みを初期化する必要があった
* エポックのループが必要だった     
* ミニバッチ学習
* 最適化手法による重みの更新
* 全結合層→活性化関数が必要
* 各層にはforwardpropagationとbackpropagationがある．
* ループの最後はsoftmax関数を通った．
* 学習と推定

# 【問題２】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。
* fit部分はwithブロックにまとめられている．
* 逆ポーランド記法
#### 《サンプルコード》

In [1]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]     #要素数
n_samples = X_train.shape[0]   #サンプル数
n_classes = 1

# 計算グラフに渡す引数の形を決める
#placeholderで空箱を作る
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワークの構造を作成
    """
    # 各層ごとの重みとバイアスの宣言
    # Variableでグローバル変数（値を更新したいものなどを入れる）
    #tf.random_normalは正規分布に従う乱数の生成
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),   
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])   # (X × w)+ bの意味
    layer_1 = tf.nn.relu(layer_1)   #出力をReLUに通す
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output


# ネットワーク構造の読み込み 
#def exsample_net()関数にXを入れる（返り値はlayer_output）
logits = example_net(X)

# 目的関数（tf.nn.sigmoid_cross_entropy_with_logitsでクロスエントロピーを求め，tf.reduce_meanでその平均をとっている）
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法（adam）
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#前側Yは０か１，後ろ側は確率．それをsignを通してー１，０，１の値にし，等しいかどうか比べている
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
#グローバル変数(variables)とみなしたもののメモリを確保する
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})   #feed_dictに計算したいデータを渡す
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples   #エポックごとのloss平均を出す
        total_acc /= n_samples    #エポックごとのaccuracyの平均
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

W0909 21:37:01.772736 18280 deprecation.py:323] From C:\Users\miyas\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0, loss : 21.8213, val_loss : 14.3145, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 1.7806, val_loss : 5.3711, acc : 0.750, val_acc : 0.625
Epoch 2, loss : 1.0733, val_loss : 6.0348, acc : 0.750, val_acc : 0.688
Epoch 3, loss : 1.7004, val_loss : 5.7219, acc : 0.750, val_acc : 0.625
Epoch 4, loss : 0.0001, val_loss : 4.3085, acc : 1.000, val_acc : 0.812
Epoch 5, loss : 0.1020, val_loss : 3.6369, acc : 1.000, val_acc : 0.688
Epoch 6, loss : 0.0000, val_loss : 1.5196, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 1.0956, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 0.8045, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 0.4127, acc : 1.000, val_acc : 0.938
test_acc : 0.800


# ２．他のデータセットへの適用
これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。
* Iris（3種類全ての目的変数を使用）
* House Prices                            
どのデータセットもtrain, val, testの3種類に分けて使用してください。
# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。                
2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。                        
### 《ヒント》                        
以下の2箇所は2クラス分類特有の処理です。                      
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))                      
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))                    
### メモ
* 多クラス分類の場合，ラベルはOne-Hot表現にする
* クロスエントロピー誤差は**tf.nn.softmax_cross_entropy_with_logits**になる
* (多クラス分類のロジスティック回帰の場合，活性化関数にシグモイドは使えず，softmax関数を用いる)                      
https://www.renom.jp/ja/notebooks/tutorial/basic_algorithm/lossfunction/notebook.html

In [2]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis] #整数型に変えて，１列に変換

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 正解データをOne-Hot表現に
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='error', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, :])
y_test_one_hot = enc.transform(y_test[:, :])

# さらにtrainとvalに分割
X_train, X_val, y_train_OH, y_val_OH = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

#shapeの確認
print('X_train',X_train.shape)
print('y_train',y_train.shape)
print('y_train_OH',y_train_OH.shape)

X_train (96, 4)
y_train (120, 1)
y_train_OH (96, 3)


C:\Users\miyas\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3   #One-Hot表現にしたので，３

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_OH, batch_size=batch_size)  #yはOne-Hotにしたものを

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数 （3値分類用のクロスエントロピー）
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#tf.arg_max(入力, argmaxしたい軸)　入力されたｙを列方向に見て，一番大きい値の列番号を返すことで，どのラベルに入るかが分かる
correct_pred = tf.equal(tf.arg_max(Y, 1), tf.arg_max(tf.sigmoid(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_OH})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

W0909 21:37:02.224319 18280 deprecation.py:323] From <ipython-input-3-6bc7f913a978>:50: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0909 21:37:02.375881 18280 deprecation.py:323] From <ipython-input-3-6bc7f913a978>:57: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


Epoch 0, loss : 149.8445, val_loss : 110.0271, acc : 0.167, val_acc : 0.375
Epoch 1, loss : 0.0000, val_loss : 7.0509, acc : 1.000, val_acc : 0.708
Epoch 2, loss : 0.0000, val_loss : 13.7986, acc : 1.000, val_acc : 0.708
Epoch 3, loss : 0.0001, val_loss : 2.6300, acc : 1.000, val_acc : 0.958
Epoch 4, loss : 2.0524, val_loss : 5.0138, acc : 0.833, val_acc : 0.875
Epoch 5, loss : 0.0000, val_loss : 2.9013, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.0026, val_loss : 3.8839, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 1.9631, acc : 1.000, val_acc : 0.958
Epoch 8, loss : 0.0000, val_loss : 2.1305, acc : 1.000, val_acc : 0.958
Epoch 9, loss : 0.0000, val_loss : 2.2735, acc : 1.000, val_acc : 0.958
test_acc : 1.000


# 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。                                      
train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。分類問題と回帰問題の違いを考慮してください。

In [4]:
df = pd.read_csv('C:/Users/miyas/kaggle/train.csv')
X = df.loc[:,['GrLivArea','YearBuilt']].values
X_log = np.log(X)
y = df.SalePrice.values
y_log = np.log(y).reshape(-1,1)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_log, y_log, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


"""#標準化
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_std = sc_X.fit_transform(X_train)
X_test_std = sc_X.transform(X_test)
y_train_std = sc_y.fit_transform(y_train[:,np.newaxis])
y_test_std = sc_y.transform(y_test[:,np.newaxis])

# さらにtrainとvalに分割
X_train_std, X_val_std, y_train_std, y_val_std = train_test_split(X_train_std, y_train_std, test_size=0.2, random_state=0)
y_train_std.shape
"""


'#標準化\nfrom sklearn.preprocessing import StandardScaler\nsc_X = StandardScaler()\nsc_y = StandardScaler()\nX_train_std = sc_X.fit_transform(X_train)\nX_test_std = sc_X.transform(X_test)\ny_train_std = sc_y.fit_transform(y_train[:,np.newaxis])\ny_test_std = sc_y.transform(y_test[:,np.newaxis])\n\n# さらにtrainとvalに分割\nX_train_std, X_val_std, y_train_std, y_val_std = train_test_split(X_train_std, y_train_std, test_size=0.2, random_state=0)\ny_train_std.shape\n'

In [5]:
"""
TensorFlowで実装したニューラルネットワークを使い，BostonHousePriceデータセットのSalePriceを予測する
"""
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]    #特徴量の数
print(n_input)
n_samples = X_train.shape[0]  #サンプル数
print(n_samples)
n_classes = 1                 #ターゲットの数

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # 活性化関数を通っているが，たぶん何もしていない？（マイナスのものは０に，プラスのものはそのまま流しているから）
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数(MSE)
loss_op = tf.reduce_mean(tf.square(Y - logits))

#最適化手法　(以下２行のどちらかで，フォワードとバックどちらもやってくれる)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = logits
# 指標値計算（決定係数R2）
R2 = 1 - (tf.reduce_sum(tf.square(Y - logits)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, R2], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, R2], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(R2, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

2
934
Epoch 0, loss : 16.9367, val_loss : 21.2110, acc : -474.088, val_acc : -135.029
Epoch 1, loss : 11.4250, val_loss : 15.6355, acc : -319.481, val_acc : -99.273
Epoch 2, loss : 13.1370, val_loss : 11.9663, acc : -367.502, val_acc : -75.742
Epoch 3, loss : 6.2132, val_loss : 5.8605, acc : -173.285, val_acc : -36.584
Epoch 4, loss : 1.2735, val_loss : 1.5269, acc : -34.722, val_acc : -8.792
Epoch 5, loss : 0.1707, val_loss : 0.3715, acc : -3.788, val_acc : -1.382
Epoch 6, loss : 0.0598, val_loss : 0.1654, acc : -0.677, val_acc : -0.061
Epoch 7, loss : 0.0112, val_loss : 0.0625, acc : 0.685, val_acc : 0.599
Epoch 8, loss : 0.0521, val_loss : 0.0661, acc : -0.463, val_acc : 0.576
Epoch 9, loss : 0.0972, val_loss : 0.0902, acc : -1.728, val_acc : 0.422
test_acc : 0.174


# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。
3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。
スクラッチで実装したモデルの再現を目指してください。

In [6]:
# MNISTデータのダウンロード
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
# 型の変換
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# 正解データをOne-Hot表現に
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# trainデータをtrainとvalに分ける
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_OH, y_val_OH = train_test_split(X_train, y_train_one_hot, test_size=0.2)

#shapeの確認
print('y_train_shape',y_train.shape) # (60000,)
print('y_train_one_hot_shape',y_train_OH.shape) # (60000, 10)
print('y_train_one_hot_dtype:',y_train_one_hot.dtype) # float64
print('X_train_shape',X_train.shape)

Using TensorFlow backend.


y_train_shape (60000,)
y_train_one_hot_shape (48000, 10)
y_train_one_hot_dtype: float64
X_train_shape (48000, 784)


In [7]:
"""
TensorFlowで実装したニューラルネットワークを使い，MNISTデータを分類する
"""
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10   #One-Hot表現にしたので，３

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_OH, batch_size=batch_size)  #yはOne-Hotにしたものを

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#tf.arg_max(入力, argmaxしたい軸)　入力されたｙを列方向に見て，一番大きい値の列番号を返すことで，どのラベルに入るかが分かる
correct_pred = tf.equal(tf.arg_max(Y, 1), tf.arg_max(tf.sigmoid(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#１エポックで何回した回数
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_OH})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.8070, val_loss : 0.9661, acc : 0.700, val_acc : 0.776
Epoch 1, loss : 1.0778, val_loss : 0.6898, acc : 0.800, val_acc : 0.830
Epoch 2, loss : 0.3661, val_loss : 0.5141, acc : 0.900, val_acc : 0.873
Epoch 3, loss : 0.7281, val_loss : 0.3532, acc : 0.700, val_acc : 0.911
Epoch 4, loss : 0.6208, val_loss : 0.3525, acc : 0.700, val_acc : 0.913
Epoch 5, loss : 0.4500, val_loss : 0.5556, acc : 0.800, val_acc : 0.905
Epoch 6, loss : 0.6561, val_loss : 0.3422, acc : 0.800, val_acc : 0.927
Epoch 7, loss : 0.5202, val_loss : 0.3478, acc : 0.800, val_acc : 0.921
Epoch 8, loss : 0.2495, val_loss : 0.3295, acc : 0.900, val_acc : 0.926
Epoch 9, loss : 0.7255, val_loss : 0.3457, acc : 0.800, val_acc : 0.924
test_acc : 0.926


In [10]:
"""
TensorFlowで実装したニューラルネットワークを使い，MNISTデータを分類する
"""
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10   #One-Hot表現にしたので，３

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_OH, batch_size=batch_size)  #yはOne-Hotにしたものを

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#tf.arg_max(入力, argmaxしたい軸)　入力されたｙを列方向に見て，一番大きい値の列番号を返すことで，どのラベルに入るかが分かる
correct_pred = tf.equal(tf.arg_max(Y, 1), tf.arg_max(tf.sigmoid(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)#１エポックで何バッチ回したか
        #エポックごとにlossを初期化
        total_loss = 0
        total_acc = 0
        total_val_loss = 0
        total_val_acc = 0
        
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_OH})#valは重みの学習がイラナイ
            
            #１バッチごとに求まるlossを全部足していく
            total_loss += loss
            total_acc += acc
            total_val_loss += val_loss
            total_acc += val_acc
        
        #lossの保存
        #バッチ数で割ることで，１エポックの平均のlossが求まる
        total_loss /= total_batch
        total_acc /= total_batch
        total_val_loss /= total_batch
        total_val_acc /= total_batch
        
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, total_val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 5.1470, val_loss : 5.0339, acc : 0.500, val_acc : 0.729
Epoch 1, loss : 0.9946, val_loss : 0.9879, acc : 0.500, val_acc : 0.798
Epoch 2, loss : 0.6880, val_loss : 0.7018, acc : 0.600, val_acc : 0.870
Epoch 3, loss : 0.4544, val_loss : 0.4778, acc : 0.700, val_acc : 0.913
Epoch 4, loss : 0.3575, val_loss : 0.3812, acc : 0.700, val_acc : 0.911
Epoch 5, loss : 0.3388, val_loss : 0.3610, acc : 0.800, val_acc : 0.918
Epoch 6, loss : 0.3146, val_loss : 0.3601, acc : 0.800, val_acc : 0.915
Epoch 7, loss : 0.2919, val_loss : 0.3403, acc : 0.600, val_acc : 0.916
Epoch 8, loss : 0.2853, val_loss : 0.3392, acc : 0.900, val_acc : 0.924
Epoch 9, loss : 0.2793, val_loss : 0.3344, acc : 0.800, val_acc : 0.918
test_acc : 0.920
